In [18]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import tensorflow as tf

### 1] ModelCheckpoint

#### Description:
The `ModelCheckpoint` callback is used to **save the model or weights at specified intervals, typically at the end of an epoch**. **It is particularly useful for saving the best version of the model (based on validation performance) during training**.

#### Use Case:
- **Saving the Best Model**: When training a neural network, it is common to save the model that performs best on the validation set to avoid overfitting.
- **Periodic Saving**: To periodically save the model during training in case of interruption, which allows resuming training from the last checkpoint.
- **Tracking Progress**: To save models at different stages of training for later comparison.

#### Syntax:
```python
tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
    options=None,
    **kwargs
)
```

#### Parameters:
- `filepath`: String, path where the model file will be saved. Can contain placeholders such as `{epoch}` and `{val_loss:.2f}`.
- `monitor`: Quantity to be monitored. Default is `'val_loss'`.
- `verbose`: Verbosity mode, 0 or 1.
- `save_best_only`: If `True`, the latest best model according to the quantity monitored will not be overwritten.
- `save_weights_only`: If `True`, then only the model's weights will be saved (`model.save_weights(filepath)`), else the full model is saved (`model.save(filepath)`).
- `mode`: One of `{'auto', 'min', 'max'}`. If `save_best_only=True`, the decision to overwrite the current save file is made based on the maximization or minimization of the monitored quantity. For `val_acc`, this would be `max`, for `val_loss` this would be `min`, etc. In `auto` mode, the direction is inferred automatically from the name of the monitored quantity.
- `save_freq`: `'epoch'` or integer. When using `'epoch'`, the callback saves the model after each epoch. When using an integer, the callback saves the model at end of this many batches.
- `options`: Optional `tf.train.CheckpointOptions` object.

In [14]:
#model
model=tf.keras.Sequential([
    #input layer
    tf.keras.layers.Dense(64,activation='relu',input_shape=(32,)),
    #hidden layer
    tf.keras.layers.Dense(128,activation='relu'),
    #ouput since it has classes from 0 to 9 totally 10 classes
    tf.keras.layers.Dense(10,activation='softmax')
])

#o/p class are integer label --> so using sparse categorrical cross entropy
#o/p metrics for classificatio  --> accuracy / precision here we using accuracy
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [15]:
#Dummy data

#training set
x_train=tf.random.normal((1000,32))
y_train=tf.random.uniform((1000,),maxval=10,dtype=tf.int32)

#testing set
x_val=tf.random.normal((200,32))
y_val=tf.random.uniform((200,),maxval=10,dtype=tf.int32)

In [16]:
# ModelCheckpoint callback
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.h5',  # File path to save the model
    monitor='val_loss',# Quantity to monitor, e.g., validation loss
    verbose=1 ,                 # Verbosity mode: 0 (silent), 1 (progress bar), or 2 (one line per epoch)
    save_best_only=True,       # Save only the best model, not after every epoch
    save_weights_only=False,   # Save the whole model (True to save only weights)
    mode='min',                # Mode: 'min' because we want to minimize validation loss
    save_freq='epoch',         # Save the model after every epoch
    
)

In [19]:
history=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,callbacks=[checkpoint_cb])

Epoch 1/10
30/32 [===========================>..] - ETA: 0s - loss: 1.8488 - accuracy: 0.4167
Epoch 1: val_loss did not improve from 2.36599
32/32 [==============================] - 0s 12ms/step - loss: 1.8483 - accuracy: 0.4110 - val_loss: 2.5155 - val_accuracy: 0.1050
Epoch 2/10
27/32 [========================>.....] - ETA: 0s - loss: 1.8113 - accuracy: 0.4317
Epoch 2: val_loss did not improve from 2.36599
32/32 [==============================] - 0s 9ms/step - loss: 1.8011 - accuracy: 0.4380 - val_loss: 2.5414 - val_accuracy: 0.1000
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 1.7446 - accuracy: 0.4601
Epoch 3: val_loss did not improve from 2.36599
32/32 [==============================] - 0s 9ms/step - loss: 1.7502 - accuracy: 0.4530 - val_loss: 2.5700 - val_accuracy: 0.1000
Epoch 4/10
32/32 [==============================] - ETA: 0s - loss: 1.7034 - accuracy: 0.4870
Epoch 4: val_loss did not improve from 2.36599
32/32 [==============================] - 0s 10ms

### 2] EarlyStopping

#### Description:
The `EarlyStopping` callback in Keras is **used to stop training when a monitored metric has stopped improving**. It helps to **prevent overfitting by halting the training process when the model performance on a validation set does not improve** for a specified number of epochs.

#### Use Case:
- **Preventing Overfitting**: Stops training when the model starts to overfit the training data and the performance on the validation set deteriorates.
- **Resource Efficiency**: Saves computational resources by halting the training process early instead of running for a fixed number of epochs.
- **Improving Model Generalization**: Helps in achieving better generalization by stopping at the right point before overfitting occurs.

#### Syntax:
```python
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)
```

#### Parameters:
- `monitor`: Quantity to be monitored. Default is `'val_loss'`.
- `min_delta`: **Minimum change in the monitored quantity to qualify as an improvement**, i.e., an absolute change of less than `min_delta` will count as no improvement.
- `patience`: Number of epochs with no improvement after which training will be stopped.
- `verbose`: Verbosity mode, 0 or 1.
- `mode`: One of `{'auto', 'min', 'max'}`. In `min` mode, training will stop when the `val_loss` monitored has stopped decreasing; in `max` mode it will stop when the `val_acc` monitored has stopped increasing; in `auto` mode, the direction is inferred automatically from the name of the monitored quantity.
- `baseline`: Baseline value for the monitored quantity. Training will stop if the model doesn't show improvement over the baseline.**For example, if you know that a validation accuracy below 80% is not acceptable, you can set baseline=0.8**
- `restore_best_weights`: Whether to restore model weights from the epoch with the best value of the monitored quantity. If `False`, the model weights obtained at the last step of training are used.
- `start_from_epoch`: Number of epochs to wait before starting to monitor improvement.

In [20]:
# EarlyStopping callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',        # Monitoring validation loss
    patience=3,                # Number of epochs with no improvement after which training will be stopped
    verbose=1,                 # Verbosity mode
    mode='min',                # Mode 'min' since we are monitoring loss
    baseline=0.4,              # Training will stop if val_loss does not improve over 0.4
    min_delta=0.01,            # Minimum change to qualify as an improvement
    start_from_epoch=10,       # Start monitoring from the 10th epoch
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [22]:
history=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=500,callbacks=[early_stopping_cb])

Epoch 1/500
32/32 [==============================] - 0s 12ms/step - loss: 0.8472 - accuracy: 0.8350 - val_loss: 3.2616 - val_accuracy: 0.1050
Epoch 2/500
32/32 [==============================] - 0s 10ms/step - loss: 0.8058 - accuracy: 0.8420 - val_loss: 3.3380 - val_accuracy: 0.0800
Epoch 3/500
32/32 [==============================] - 0s 12ms/step - loss: 0.7622 - accuracy: 0.8630 - val_loss: 3.4014 - val_accuracy: 0.0950
Epoch 4/500
32/32 [==============================] - 0s 9ms/step - loss: 0.7245 - accuracy: 0.8740 - val_loss: 3.4583 - val_accuracy: 0.0950
Epoch 5/500
32/32 [==============================] - 0s 9ms/step - loss: 0.6807 - accuracy: 0.8940 - val_loss: 3.4841 - val_accuracy: 0.0900
Epoch 6/500
32/32 [==============================] - 0s 9ms/step - loss: 0.6438 - accuracy: 0.9050 - val_loss: 3.5692 - val_accuracy: 0.0950
Epoch 7/500
32/32 [==============================] - 0s 8ms/step - loss: 0.6047 - accuracy: 0.9140 - val_loss: 3.6186 - val_accuracy: 0.1000
Epoch 8/50

### 3] ReduceLROnPlateau

The `ReduceLROnPlateau` callback in Keras is **used to reduce the learning rate when a monitored metric has stopped improving**. This is particularly **useful for fine-tuning the learning process**, ensuring that the model can converge to a better solution without getting stuck at a suboptimal learning rate.


#### Syntax

```python
tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=10,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)
```

- **Key Parameters**:
  - `monitor`: The metric to be monitored (e.g., `val_loss`).
  - `factor`: The factor by which the learning rate will be reduced (new_lr = lr * factor).
  - `patience`: Number of epochs with no improvement after which the learning rate will be reduced.
  - `verbose`: Verbosity mode (0 or 1).
  - `mode`: One of `{'auto', 'min', 'max'}`. In `min` mode, the learning rate will be reduced when the quantity monitored has stopped decreasing; in `max` mode, it will be reduced when the quantity monitored has stopped increasing.
  - `min_delta`: Threshold for measuring the new optimum; to only focus on significant changes.
  - `cooldown`: Number of epochs to wait before resuming normal operation after the learning rate has been reduced.
  - `min_lr`: Lower bound on the learning rate.

In [28]:
model=tf.keras.Sequential([
    #input layer
    tf.keras.layers.Dense(64,activation='relu',input_shape=(32,)),
    #hidden layer
    tf.keras.layers.Dense(128,activation='relu'),
    #ouput
    tf.keras.layers.Dense(10,activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [29]:
#dummy data

#train
x_train=tf.random.normal((1000,32))
y_train=tf.random.uniform((1000,),maxval=10,dtype=tf.int32)

#validation dataset
x_val=tf.random.normal((200,32))
y_val=tf.random.uniform((200,),maxval=10,dtype=tf.int32)

In [30]:
# ReduceLROnPlateau callback
reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',    # Monitoring validation loss
    factor=0.1,            # Factor by which the learning rate will be reduced. new_lr = lr * factor
    patience=5,            # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,             # Verbosity mode
    mode='min',            # Mode 'min' because we want to minimize validation loss
    min_delta=0.001,       # Threshold for measuring the new optimum
    cooldown=2,            # Number of epochs to wait before resuming normal operation after LR is reduced
    min_lr=0.00001         # Lower bound on the learning rate
)

In [31]:
history=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=30,callbacks=[reduce_lr_cb])

Epoch 1/30
32/32 [==============================] - 2s 21ms/step - loss: 2.3466 - accuracy: 0.0900 - val_loss: 2.3055 - val_accuracy: 0.1100 - lr: 0.0010
Epoch 2/30
32/32 [==============================] - 0s 9ms/step - loss: 2.2568 - accuracy: 0.1540 - val_loss: 2.3162 - val_accuracy: 0.1250 - lr: 0.0010
Epoch 3/30
32/32 [==============================] - 0s 9ms/step - loss: 2.2011 - accuracy: 0.1870 - val_loss: 2.3349 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 4/30
32/32 [==============================] - 0s 10ms/step - loss: 2.1513 - accuracy: 0.2430 - val_loss: 2.3512 - val_accuracy: 0.1050 - lr: 0.0010
Epoch 5/30
32/32 [==============================] - 0s 9ms/step - loss: 2.1092 - accuracy: 0.2750 - val_loss: 2.3664 - val_accuracy: 0.1050 - lr: 0.0010
Epoch 6/30
31/32 [============================>.] - ETA: 0s - loss: 2.0595 - accuracy: 0.2974
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
32/32 [==============================] - 0s 9ms/step - loss: 2

### 4] CSV logger


- **Purpose**: To log the training metrics (loss and metrics) to a CSV file after each epoch during training.
- **Use Case**: It helps in tracking the training progress, analyzing the trends of loss and metrics over epochs, and comparing different training runs.


### Benefits

- Enables easy tracking and comparison of training metrics across epochs.
- Facilitates visual analysis and debugging of model training.

The CSVLogger is a simple yet effective tool for managing and reviewing model training progress in TensorFlow/Keras projects.

In [32]:
csv_logger = tf.keras.callbacks.CSVLogger('training.log')

In [33]:
history=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,callbacks=[csv_logger])

Epoch 1/10
32/32 [==============================] - 0s 12ms/step - loss: 1.9677 - accuracy: 0.3560 - val_loss: 2.4073 - val_accuracy: 0.0950
Epoch 2/10
32/32 [==============================] - 0s 10ms/step - loss: 1.9672 - accuracy: 0.3570 - val_loss: 2.4074 - val_accuracy: 0.0950
Epoch 3/10
32/32 [==============================] - 0s 9ms/step - loss: 1.9667 - accuracy: 0.3550 - val_loss: 2.4077 - val_accuracy: 0.0950
Epoch 4/10
32/32 [==============================] - 0s 9ms/step - loss: 1.9663 - accuracy: 0.3570 - val_loss: 2.4080 - val_accuracy: 0.0900
Epoch 5/10
32/32 [==============================] - 0s 9ms/step - loss: 1.9658 - accuracy: 0.3580 - val_loss: 2.4082 - val_accuracy: 0.0900
Epoch 6/10
32/32 [==============================] - 0s 9ms/step - loss: 1.9654 - accuracy: 0.3600 - val_loss: 2.4084 - val_accuracy: 0.0900
Epoch 7/10
32/32 [==============================] - 0s 8ms/step - loss: 1.9649 - accuracy: 0.3600 - val_loss: 2.4087 - val_accuracy: 0.0900
Epoch 8/10
32/32 [

In [35]:
import pandas as pd

In [36]:
#Reading the stored log file
log=pd.read_csv('training.log')

In [37]:
log

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.356,1.967657,0.095,2.407253
1,1,0.357,1.967200,0.095,2.407428
2,2,0.355,1.966738,0.095,2.407713
3,3,0.357,1.966263,0.090,2.408009
4,4,0.358,1.965820,0.090,2.408202
5,5,0.360,1.965369,0.090,2.408405
6,6,0.360,1.964866,0.090,2.408714
7,7,0.362,1.964439,0.090,2.409068
8,8,0.361,1.963970,0.090,2.409299
9,9,0.361,1.963524,0.090,2.409408


### 5] TerminateOnNaN


- **Purpose**: To terminate the training process if the loss becomes NaN during training.
- **Use Case**: This callback is essential for ensuring the stability and reliability of training, especially when dealing with large datasets or complex models where numerical instability might occur.

##### Example Scenario

Suppose during training, **the loss function starts producing NaN values due to some numerical instability in the model's computations. Without the TerminateOnNaN callback, the training process would continue, potentially leading to inaccurate model updates and wasted computational resources.** By using this callback, TensorFlow/Keras can automatically halt the training process upon detecting such issues, allowing you to investigate and address the underlying causes.

##### Benefits

- Ensures training stability by stopping on encountering NaN values in the loss.
- Helps in debugging model training issues related to numerical instability.
- Prevents wasted computational resources and time by halting training early when issues arise.

In summary, the TerminateOnNaN callback is a critical tool for maintaining the integrity and reliability of model training in TensorFlow/Keras, ensuring that the training process halts promptly upon detecting NaN values in the loss function.

In [38]:
# TerminateOnNaN callback
terminate_on_nan = tf.keras.callbacks.TerminateOnNaN()

In [39]:
history=model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=40,callbacks=[terminate_on_nan])

Epoch 1/40
32/32 [==============================] - 0s 12ms/step - loss: 1.9631 - accuracy: 0.3630 - val_loss: 2.4097 - val_accuracy: 0.0900
Epoch 2/40
32/32 [==============================] - 0s 9ms/step - loss: 1.9626 - accuracy: 0.3630 - val_loss: 2.4099 - val_accuracy: 0.0900
Epoch 3/40
32/32 [==============================] - 0s 9ms/step - loss: 1.9621 - accuracy: 0.3640 - val_loss: 2.4102 - val_accuracy: 0.0900
Epoch 4/40
32/32 [==============================] - 0s 9ms/step - loss: 1.9617 - accuracy: 0.3650 - val_loss: 2.4103 - val_accuracy: 0.0900
Epoch 5/40
32/32 [==============================] - 0s 9ms/step - loss: 1.9612 - accuracy: 0.3660 - val_loss: 2.4105 - val_accuracy: 0.0900
Epoch 6/40
32/32 [==============================] - 0s 9ms/step - loss: 1.9608 - accuracy: 0.3660 - val_loss: 2.4107 - val_accuracy: 0.0850
Epoch 7/40
32/32 [==============================] - 0s 12ms/step - loss: 1.9603 - accuracy: 0.3650 - val_loss: 2.4108 - val_accuracy: 0.0850
Epoch 8/40
32/32 [

In [40]:
#since no nan occur so it runs for all epochs

### 6] Learning Rate Scheduler

A learning rate scheduler in TensorFlow/Keras is a callback used to adjust the learning rate during training. Adjusting the learning rate can help improve convergence and model performance.

#### Description

- **Purpose**: To dynamically change the learning rate based on the epoch or other criteria.
- **Use Case**: Often used in training deep learning models to start with a higher learning rate for faster convergence and gradually reduce it to fine-tune the model.

#### Types of Learning Rate Schedulers

1. **Exponential Decay**: Reduces the learning rate by a factor of the decay rate every certain number of epochs.
2. **Step Decay**: Reduces the learning rate by a factor every few epochs.
3. **Polynomial Decay**: Reduces the learning rate following a polynomial function.
4. **Inverse Time Decay**: Reduces the learning rate as a function of the inverse of time.



#### Explanation of Parameters

- **`initial_learning_rate`**: The starting learning rate.
- **`decay_steps`**: The number of steps after which the learning rate decays.
       **Example: If decay_steps = 1000, it means that the learning rate will remain constant for the first 1000 epochs.**
- **`decay_rate`**: The factor by which the learning rate decays.
       **If decay_rate = 0.96, it means that after every decay_steps, the learning rate will be multiplied by 0.96.**
       
- **`staircase`**: 
                  **When staircase=True:** The learning rate decreases in discrete steps after every decay_steps. This means that       the learning rate remains constant until the end of each decay_steps interval, and then it drops abruptly.


                  **When staircase=False (default behavior):** The learning rate decreases smoothly and continuously with every         epoch, according to the exponential decay formula specified.
- **`end_learning_rate`**: The learning rate after all decay steps are completed.
- **`power`**: The power of the polynomial for polynomial decay.

#### Use Case

- **Exponential Decay**: When you want a smooth decay of learning rate over time.
- **Step Decay**: When you want to reduce the learning rate at specific intervals (steps).
- **Polynomial Decay**: When you want a custom decay rate following a polynomial function.
- **Inverse Time Decay**: When you want the learning rate to decay inversely with time.

These scheduling strategies help in adjusting the learning rate dynamically, improving the model's performance and convergence during training.

#### Exponential decay - eg

In [41]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

# Sample model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(32,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dummy data
x_train = tf.random.normal((1000, 32))
y_train = tf.random.uniform((1000,), maxval=10, dtype=tf.int32)
x_val = tf.random.normal((200, 32))
y_val = tf.random.uniform((200,), maxval=10, dtype=tf.int32)

# Training
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50
)


Epoch 1/50
32/32 [==============================] - 2s 25ms/step - loss: 2.3745 - accuracy: 0.1010 - val_loss: 2.3192 - val_accuracy: 0.0850
Epoch 2/50
32/32 [==============================] - 0s 10ms/step - loss: 2.2791 - accuracy: 0.1500 - val_loss: 2.3022 - val_accuracy: 0.0800
Epoch 3/50
32/32 [==============================] - 0s 9ms/step - loss: 2.2281 - accuracy: 0.1760 - val_loss: 2.3172 - val_accuracy: 0.1150
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 2.1808 - accuracy: 0.2200 - val_loss: 2.3300 - val_accuracy: 0.1050
Epoch 5/50
32/32 [==============================] - 0s 11ms/step - loss: 2.1416 - accuracy: 0.2500 - val_loss: 2.3636 - val_accuracy: 0.1000
Epoch 6/50
32/32 [==============================] - 0s 10ms/step - loss: 2.1024 - accuracy: 0.2650 - val_loss: 2.3554 - val_accuracy: 0.1000
Epoch 7/50
32/32 [==============================] - 0s 12ms/step - loss: 2.0610 - accuracy: 0.2760 - val_loss: 2.3559 - val_accuracy: 0.0950
Epoch 8/50
32/

#### Step decay - eg

In [42]:
def step_decay(epoch):
    initial_lr = 0.1
    drop = 0.5 # Factor by which the learning rate is reduced
    epochs_drop = 10 #the learning rate remains constant at initial_lr for 10 epochs as mentioned here
    lr = initial_lr * (drop ** ((epoch) // epochs_drop))
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(step_decay)

# Sample model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(32,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dummy data
x_train = tf.random.normal((1000, 32))
y_train = tf.random.uniform((1000,), maxval=10, dtype=tf.int32)
x_val = tf.random.normal((200, 32))
y_val = tf.random.uniform((200,), maxval=10, dtype=tf.int32)

# Training with Step Decay
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    callbacks=[lr_scheduler]
)


Epoch 1/50
32/32 [==============================] - 3s 22ms/step - loss: 2.4287 - accuracy: 0.0970 - val_loss: 2.3499 - val_accuracy: 0.1100 - lr: 0.1000
Epoch 2/50
32/32 [==============================] - 0s 9ms/step - loss: 2.3199 - accuracy: 0.1000 - val_loss: 2.3224 - val_accuracy: 0.1150 - lr: 0.1000
Epoch 3/50
32/32 [==============================] - 0s 9ms/step - loss: 2.3085 - accuracy: 0.1150 - val_loss: 2.3095 - val_accuracy: 0.0650 - lr: 0.1000
Epoch 4/50
32/32 [==============================] - 0s 8ms/step - loss: 2.3051 - accuracy: 0.1180 - val_loss: 2.3652 - val_accuracy: 0.0650 - lr: 0.1000
Epoch 5/50
32/32 [==============================] - 0s 9ms/step - loss: 2.3196 - accuracy: 0.0910 - val_loss: 2.3418 - val_accuracy: 0.0650 - lr: 0.1000
Epoch 6/50
32/32 [==============================] - 0s 11ms/step - loss: 2.3079 - accuracy: 0.0870 - val_loss: 2.3328 - val_accuracy: 0.1150 - lr: 0.1000
Epoch 7/50
32/32 [==============================] - 0s 10ms/step - loss: 2.3042 

#### Polynomial decay

In [43]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate,
    decay_steps=100000,
    end_learning_rate=0.01,
    power=1.0
)

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

# Sample model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(32,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dummy data
x_train = tf.random.normal((1000, 32))
y_train = tf.random.uniform((1000,), maxval=10, dtype=tf.int32)
x_val = tf.random.normal((200, 32))
y_val = tf.random.uniform((200,), maxval=10, dtype=tf.int32)

# Training
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50
)


Epoch 1/50
32/32 [==============================] - 2s 21ms/step - loss: 2.3607 - accuracy: 0.1140 - val_loss: 2.3280 - val_accuracy: 0.1050
Epoch 2/50
32/32 [==============================] - 0s 8ms/step - loss: 2.2712 - accuracy: 0.1440 - val_loss: 2.3583 - val_accuracy: 0.0850
Epoch 3/50
32/32 [==============================] - 0s 8ms/step - loss: 2.2238 - accuracy: 0.1730 - val_loss: 2.3280 - val_accuracy: 0.1150
Epoch 4/50
32/32 [==============================] - 0s 9ms/step - loss: 2.1798 - accuracy: 0.2010 - val_loss: 2.3347 - val_accuracy: 0.1400
Epoch 5/50
32/32 [==============================] - 0s 8ms/step - loss: 2.1385 - accuracy: 0.2380 - val_loss: 2.3896 - val_accuracy: 0.0850
Epoch 6/50
32/32 [==============================] - 0s 9ms/step - loss: 2.0936 - accuracy: 0.2650 - val_loss: 2.3600 - val_accuracy: 0.0850
Epoch 7/50
32/32 [==============================] - 0s 9ms/step - loss: 2.0509 - accuracy: 0.2880 - val_loss: 2.3630 - val_accuracy: 0.0950
Epoch 8/50
32/32 [=

#### Inverse time decay

In [44]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate,
    decay_steps=1.0,
    decay_rate=0.5,
    staircase=True
)

optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

# Sample model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(32,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Dummy data
x_train = tf.random.normal((1000, 32))
y_train = tf.random.uniform((1000,), maxval=10, dtype=tf.int32)
x_val = tf.random.normal((200, 32))
y_val = tf.random.uniform((200,), maxval=10, dtype=tf.int32)

# Training
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50
)


Epoch 1/50
32/32 [==============================] - 2s 28ms/step - loss: 2.3939 - accuracy: 0.0900 - val_loss: 2.3562 - val_accuracy: 0.0700
Epoch 2/50
32/32 [==============================] - 0s 10ms/step - loss: 2.3629 - accuracy: 0.0980 - val_loss: 2.3502 - val_accuracy: 0.0800
Epoch 3/50
32/32 [==============================] - 0s 10ms/step - loss: 2.3560 - accuracy: 0.1020 - val_loss: 2.3470 - val_accuracy: 0.0850
Epoch 4/50
32/32 [==============================] - 0s 9ms/step - loss: 2.3519 - accuracy: 0.1040 - val_loss: 2.3450 - val_accuracy: 0.0850
Epoch 5/50
32/32 [==============================] - 0s 10ms/step - loss: 2.3493 - accuracy: 0.1090 - val_loss: 2.3437 - val_accuracy: 0.0800
Epoch 6/50
32/32 [==============================] - 0s 11ms/step - loss: 2.3472 - accuracy: 0.1100 - val_loss: 2.3427 - val_accuracy: 0.0800
Epoch 7/50
32/32 [==============================] - 0s 9ms/step - loss: 2.3455 - accuracy: 0.1110 - val_loss: 2.3418 - val_accuracy: 0.0800
Epoch 8/50
32/3